In [3]:
import numpy as np
import pandas as pd

#plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

#stats
from scipy import stats
from scipy.stats import norm, skew

#missing data
from sklearn.impute import SimpleImputer

#classifiers
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

#metrics
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

import random
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

#warnings
import warnings
warnings.filterwarnings('ignore')

import time

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [31]:
test.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,3236.000000,0.34875,10.2006,0.0000,0.0,418564.00000,4.185640e+05,4.185640e+05,5.407100e+05,0.0,...,108.209648,0.825000,1.01,0.800000,0.0,0.0,0,301544.001781,0.000000,0.011221
1,3284.000000,1.27360,2.9606,9.0198,0.0,0.00000,9.858816e+06,4.901400e+04,1.510098e+06,18308285.0,...,18.841500,0.507694,4.04,0.623248,1.0,0.0,1,301544.001781,0.504974,0.043525
2,3492.284404,0.27505,0.0600,0.0000,0.0,626303.61796,6.797033e+06,2.654142e+06,1.350593e+07,0.0,...,108.209648,0.369684,0.00,0.566219,0.0,0.0,0,301544.001781,0.000000,674984.349339
3,3232.000000,0.28505,2.8032,0.0000,0.0,0.00000,4.738020e+05,4.738020e+05,1.724437e+06,493641.0,...,108.209648,0.916663,2.02,0.464224,0.0,0.0,0,90.163742,0.788809,0.104029
4,3466.000000,2.09545,0.8318,2.5182,0.0,19839.00000,1.150662e+06,1.150662e+06,7.860523e+06,5752921.0,...,108.209648,0.234047,23.23,0.726688,0.0,0.0,1,1303.587148,1.637733,0.163124


In [19]:
train.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [20]:
test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [8]:
def check_missing(df):
    total = df.isna().sum()
    percentage = (df.isna().sum()/df.shape[0])*100
    missing = pd.concat([total, percentage], axis=1, keys=['Total', 'percent (%)'])
    return np.transpose(missing)

In [18]:
check_missing(train)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
percent (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train.shape, test.shape

((56000, 52), (24000, 51))

In [11]:
#outlier checks in number
train[train['form_field28'] >= 50000].shape, train[train['form_field6'] >= 4e7].shape, train[train['form_field7'] >= 1.5e9].shape, 

((15407, 52), (1, 52), (1, 52))

In [12]:
#remove some outliers
train = train[train['form_field28'] <= 50000].reset_index(drop=True)

In [13]:
train.shape

(40238, 52)

In [14]:
def target_flow(train):
    defaulters = train[train['default_status'] == 'yes'].shape[0]
    total = train.shape[0]
    percent = (defaulters/total)*100
    return f'Defaulters in percent is {np.round(percent, decimals=2)}'

In [15]:
target_flow(train)

'Defaulters in percent is 30.22'

In [16]:
train['form_field1'].fillna(train['form_field1'].mean(), inplace=True)
train['form_field2'].fillna(0, inplace=True)
train['form_field3'].fillna(0, inplace=True)
train['form_field4'].fillna(0, inplace=True)
train['form_field5'].fillna(0, inplace=True)
train['form_field6'].fillna(train['form_field6'].mean(), inplace=True)
train['form_field7'].fillna(train['form_field7'].mean(), inplace=True)
train['form_field8'].fillna(train['form_field8'].mean(), inplace=True)
train['form_field9'].fillna(train['form_field9'].mean(), inplace=True)
train['form_field10'].fillna(train['form_field10'].mean(), inplace=True)
train['form_field11'].fillna(train['form_field11'].mean(), inplace=True)
train['form_field12'].fillna(train['form_field12'].mean(), inplace=True)
train['form_field13'].fillna(train['form_field13'].mean(), inplace=True)
train['form_field15'].fillna(train['form_field15'].mean(), inplace=True)
train['form_field16'].fillna(0, inplace=True)
train['form_field17'].fillna(0, inplace=True)
train['form_field18'].fillna(0, inplace=True)
train['form_field19'].fillna(0, inplace=True)
train['form_field20'].fillna(0, inplace=True)
train['form_field21'].fillna(train['form_field21'].mean(), inplace=True)
train['form_field22'].fillna(train['form_field22'].mean(), inplace=True)
train['form_field23'].fillna(train['form_field23'].mean(), inplace=True)
train['form_field24'].fillna(train['form_field24'].mean(), inplace=True)
train['form_field25'].fillna(train['form_field25'].mean(), inplace=True)
train['form_field26'].fillna(train['form_field26'].mean(), inplace=True)
train['form_field27'].fillna(train['form_field27'].mean(), inplace=True)
train['form_field28'].fillna(60, inplace=True)
train['form_field29'].fillna(train['form_field29'].mean(), inplace=True)
train['form_field30'].fillna(train['form_field30'].mean(), inplace=True)
train['form_field31'].fillna(train['form_field31'].mean(), inplace=True)
train['form_field32'].fillna(train['form_field32'].mean(), inplace=True)
train['form_field33'].fillna(train['form_field33'].mean(), inplace=True)
train['form_field34'].fillna(train['form_field34'].mode()[0], inplace=True)
train['form_field35'].fillna(train['form_field35'].mode()[0], inplace=True)
train['form_field36'].fillna(train['form_field36'].mode()[0], inplace=True)
train['form_field37'].fillna(train['form_field37'].mode()[0], inplace=True)
train['form_field38'].fillna(train['form_field38'].mode()[0], inplace=True)
train['form_field39'].fillna(train['form_field39'].mode()[0], inplace=True)
train['form_field40'].fillna(train['form_field40'].mean(), inplace=True)
train['form_field41'].fillna(train['form_field41'].mean(), inplace=True)
train['form_field42'].fillna(train['form_field42'].mean(), inplace=True)
train['form_field43'].fillna(train['form_field43'].mean(), inplace=True)
train['form_field44'].fillna(train['form_field44'].mean(), inplace=True)
train['form_field45'].fillna(train['form_field45'].mode()[0], inplace=True)
train['form_field46'].fillna(train['form_field46'].mode()[0], inplace=True)
# Encoading the categorical variables
from sklearn import preprocessing
#creating label encoader
le = preprocessing.LabelEncoder()
#converting string lables into numbers
#train['# Encoading the categorical variables
from sklearn import preprocessing
#creating label encoader
le = preprocessing.LabelEncoder()
#converting string lables into numbers
train['form_field47']=le.fit_transform(train['form_field47'])
train['default_status']=le.fit_transform(train['default_status'])
train['form_field48'].fillna(train['form_field48'].mean(), inplace=True)
train['form_field49'].fillna(train['form_field49'].mean(), inplace=True)
train['form_field50'].fillna(train['form_field50'].mean(), inplace=True)

In [17]:
test['form_field1'].fillna(test['form_field1'].mean(), inplace=True)
test['form_field2'].fillna(0, inplace=True)
test['form_field3'].fillna(0, inplace=True)
test['form_field4'].fillna(0, inplace=True)
test['form_field5'].fillna(0, inplace=True)
test['form_field6'].fillna(test['form_field6'].mean(), inplace=True)
test['form_field7'].fillna(test['form_field7'].mean(), inplace=True)
test['form_field8'].fillna(test['form_field8'].mean(), inplace=True)
test['form_field9'].fillna(test['form_field9'].mean(), inplace=True)
test['form_field10'].fillna(test['form_field10'].mean(), inplace=True)
test['form_field11'].fillna(test['form_field11'].mean(), inplace=True)
test['form_field12'].fillna(test['form_field12'].mean(), inplace=True)
test['form_field13'].fillna(test['form_field13'].mean(), inplace=True)
test['form_field15'].fillna(test['form_field15'].mean(), inplace=True)
test['form_field16'].fillna(0, inplace=True)
test['form_field17'].fillna(0, inplace=True)
test['form_field18'].fillna(0, inplace=True)
test['form_field19'].fillna(0, inplace=True)
test['form_field20'].fillna(0, inplace=True)
test['form_field21'].fillna(test['form_field21'].mean(), inplace=True)
test['form_field22'].fillna(test['form_field22'].mean(), inplace=True)
test['form_field23'].fillna(test['form_field23'].mean(), inplace=True)
test['form_field24'].fillna(test['form_field24'].mean(), inplace=True)
test['form_field25'].fillna(test['form_field25'].mean(), inplace=True)
test['form_field26'].fillna(test['form_field26'].mean(), inplace=True)
test['form_field27'].fillna(test['form_field27'].mean(), inplace=True)
test['form_field28'].fillna(60, inplace=True)
test['form_field29'].fillna(test['form_field29'].mean(), inplace=True)
test['form_field30'].fillna(test['form_field30'].mean(), inplace=True)
test['form_field31'].fillna(test['form_field31'].mean(), inplace=True)
test['form_field32'].fillna(test['form_field32'].mean(), inplace=True)
test['form_field33'].fillna(test['form_field33'].mean(), inplace=True)
test['form_field34'].fillna(test['form_field34'].mode()[0], inplace=True)
test['form_field35'].fillna(test['form_field35'].mode()[0], inplace=True)
test['form_field36'].fillna(test['form_field36'].mode()[0], inplace=True)
test['form_field37'].fillna(test['form_field37'].mode()[0], inplace=True)
test['form_field38'].fillna(test['form_field38'].mode()[0], inplace=True)
test['form_field39'].fillna(test['form_field39'].mode()[0], inplace=True)
test['form_field40'].fillna(test['form_field40'].mean(), inplace=True)
test['form_field41'].fillna(test['form_field41'].mean(), inplace=True)
test['form_field42'].fillna(test['form_field42'].mean(), inplace=True)
test['form_field43'].fillna(test['form_field43'].mean(), inplace=True)
test['form_field44'].fillna(test['form_field44'].mean(), inplace=True)
test['form_field45'].fillna(test['form_field45'].mode()[0], inplace=True)
test['form_field46'].fillna(test['form_field46'].mode()[0], inplace=True)
# Encoading the categorical variables
from sklearn import preprocessing
#creating label encoader
le = preprocessing.LabelEncoder()
#converting string lables into numbers
#train['# Encoading the categorical variables
from sklearn import preprocessing
#creating label encoader
le = preprocessing.LabelEncoder()
#converting string lables into numbers
test['form_field47']=le.fit_transform(test['form_field47'])
test['form_field48'].fillna(test['form_field48'].mean(), inplace=True)
test['form_field49'].fillna(test['form_field49'].mean(), inplace=True)
test['form_field50'].fillna(test['form_field50'].mean(), inplace=True)

In [29]:
train.drop(['Applicant_ID','default_status'], axis=1, inplace = True)
test.drop(['Applicant_ID'], axis=1, inplace = True)

In [32]:
train['source'] = 'train'
test['source'] = 'test'

In [34]:
train1 = pd.read_csv('train.csv')
target = train1['default_status'].copy()

In [35]:
#combine both train and test for feature engineering. Also, being careful to cause data leakage (no transforms)
combo = pd.concat([train, test], axis=0)
combo.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,source
0,3436.0,0.28505,1.6560,0.0,0.000,0.000000,10689720.0,2.520720e+05,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,0,212244.246183,1.129518,0.044335,train
1,3456.0,0.67400,0.2342,0.0,0.000,0.000000,898979.0,4.975310e+05,9073814.0,2533168.0,...,0.314281,8.08,0.183584,0.0,0.0,0,349.805730,1.620483,0.322436,train
2,3276.0,0.53845,3.1510,0.0,6.282,322897.083846,956940.0,1.541810e+06,192944.0,1079864.0,...,0.162965,18.18,0.791136,0.0,0.0,0,212244.246183,1.513370,0.011640,train
3,3372.0,0.17005,0.5050,0.0,0.000,192166.000000,3044703.0,3.854990e+05,3986472.0,3621979.0,...,0.488884,2.02,0.685168,0.0,0.0,0,89.940100,0.664452,0.082729,train
4,3370.0,0.77270,1.1010,0.0,0.000,1556.000000,214728.0,2.147280e+05,1284089.0,361770.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,train


In [36]:
combo.reset_index(drop=True, inplace=True)

In [37]:
check_missing(combo)

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,source
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
percent (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#Feature Engineering - MINOR

combo['income_collateral'] = combo['form_field14'] - combo['form_field15']
combo['income_loan_repay'] = combo['form_field14'] - combo['form_field13']
combo['income_due'] = (combo.form_field14 - combo.form_field12)/10000

combo['max_min_active'] = combo.form_field8 - combo.form_field6
combo['max_active'] = combo.form_field8 - combo.form_field7


combo['edu_auto'] = combo.form_field5 - combo.form_field4
combo['auto_any'] = combo.form_field4 - combo.form_field3
combo['edu_any'] = combo.form_field5 - combo.form_field3

combo['percentage_edu_auto'] = combo.form_field40 - combo.form_field41

combo['active_lines_now'] = combo.form_field38 - combo.form_field36
combo['deliq'] = combo.form_field38 - combo.form_field39
combo['deliq_ff1'] = (combo.deliq/(combo['form_field1']))

combo['unknowns'] = ((combo.form_field48/(combo.form_field49 + 1))/10000)*combo.form_field50
combo['ps'] = (combo.income_due - combo.income_collateral)/10000

combo['tenure_days'] = combo.form_field27/(combo.form_field25 + 1)

combo['extrasI'] = combo.form_field17/combo.form_field1
combo['extrasII'] = combo.form_field14/combo.form_field1
combo['extrasIII'] = combo.form_field1*combo.form_field49

In [39]:
def check_inf(combo):
    neg_inf = -np.inf in combo
    pos_inf = np.inf in combo
    return neg_inf, pos_inf

In [40]:
check_inf(combo)

(False, False)

In [57]:
train_data = combo[combo['source'] == 'train']
test_data = combo[combo['source'] == 'test']

In [58]:
train_data['default_status'] = target

In [67]:
train_data.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,active_lines_now,deliq,deliq_ff1,unknowns,ps,tenure_days,extrasI,extrasII,extrasIII,default_status
0,3436.0,0.28505,1.6560,0.0,0.000,0.000000,10689720.0,2.520720e+05,4272776.0,11333126.0,...,-8.0,4.0,0.001164,0.441875,1581.603338,2.842934,0.000291,28048.643481,3881.024096,0
1,3456.0,0.67400,0.2342,0.0,0.000,0.000000,898979.0,4.975310e+05,9073814.0,2533168.0,...,-8.0,8.0,0.002315,0.004304,1703.089085,2.148486,0.000289,8142.774016,5600.388916,0
2,3276.0,0.53845,3.1510,0.0,6.282,322897.083846,956940.0,1.541810e+06,192944.0,1079864.0,...,-6.0,6.0,0.001832,0.098292,587.764976,1.979356,0.000000,5059.968559,4957.801012,1
3,3372.0,0.17005,0.5050,0.0,0.000,192166.000000,3044703.0,3.854990e+05,3986472.0,3621979.0,...,0.0,3.0,0.000890,0.000447,8811.518656,1.572268,0.000000,14290.443357,2240.531561,0
4,3370.0,0.77270,1.1010,0.0,0.000,1556.000000,214728.0,2.147280e+05,1284089.0,361770.0,...,0.0,24.0,0.007122,0.000184,-1284.593770,1.411082,0.000000,8350.571810,4811.994288,0


In [65]:
le = preprocessing.LabelEncoder()
#converting string lables into numbers
train_data['form_field47']=le.fit_transform(train['form_field47'])
train_data['default_status'] = train_data.default_status.map({'yes':1, 'no':0}).astype('int8')


In [66]:
train_data['default_status']

0        0
1        0
2        1
3        0
4        0
        ..
40233    0
40234    1
40235    1
40236    1
40237    0
Name: default_status, Length: 40238, dtype: int8

In [68]:
from sklearn.cluster import KMeans

In [69]:
kmeans = KMeans(n_clusters=3, max_iter=400, n_jobs=-1, random_state=3)

In [70]:
train_data['cluster'] = kmeans.fit_predict(train_data[['form_field1','form_field2', 'form_field49']])
test_data['cluster'] = kmeans.predict(test_data[['form_field1','form_field2', 'form_field49']])

In [71]:
train_data.cluster.value_counts()

0    18363
2    11420
1    10455
Name: cluster, dtype: int64

In [72]:
def cluster_map(x):
    if x == 0:
        return 'clu0'
    elif x == 1:
        return 'clu1'
    else:
        return 'clu2'

In [73]:
train_data['cluster'] = train_data.cluster.map(cluster_map)
test_data['cluster'] = test_data.cluster.map(cluster_map)

In [74]:
train_data['cluster'].sample(6)

1804     clu0
22633    clu0
6617     clu2
9137     clu0
10069    clu1
29509    clu0
Name: cluster, dtype: object

In [75]:
train_data.drop(['source'], axis=1, inplace=True)
test_data.drop(['source'], axis=1, inplace=True)

In [76]:
train_data_ = pd.get_dummies(train_data, drop_first=True)
test_data_ = pd.get_dummies(test_data, drop_first=True)

In [77]:
train_data_.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,deliq_ff1,unknowns,ps,tenure_days,extrasI,extrasII,extrasIII,default_status,cluster_clu1,cluster_clu2
0,3436.0,0.28505,1.6560,0.0,0.000,0.000000,10689720.0,2.520720e+05,4272776.0,11333126.0,...,0.001164,0.441875,1581.603338,2.842934,0.000291,28048.643481,3881.024096,0,0,0
1,3456.0,0.67400,0.2342,0.0,0.000,0.000000,898979.0,4.975310e+05,9073814.0,2533168.0,...,0.002315,0.004304,1703.089085,2.148486,0.000289,8142.774016,5600.388916,0,0,0
2,3276.0,0.53845,3.1510,0.0,6.282,322897.083846,956940.0,1.541810e+06,192944.0,1079864.0,...,0.001832,0.098292,587.764976,1.979356,0.000000,5059.968559,4957.801012,1,1,0
3,3372.0,0.17005,0.5050,0.0,0.000,192166.000000,3044703.0,3.854990e+05,3986472.0,3621979.0,...,0.000890,0.000447,8811.518656,1.572268,0.000000,14290.443357,2240.531561,0,0,0
4,3370.0,0.77270,1.1010,0.0,0.000,1556.000000,214728.0,2.147280e+05,1284089.0,361770.0,...,0.007122,0.000184,-1284.593770,1.411082,0.000000,8350.571810,4811.994288,0,0,0


In [79]:
x_train = train_data_.copy()
y_train = train_data_.default_status
x_test = test_data_

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

In [81]:
train_features, test_features, train_labels, test_labels = train_test_split(x_train,y_train,test_size=0.2,
    random_state=41)

In [82]:
correlated_features = set()
correlation_matrix = x_train.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [83]:
train_features.drop(labels=correlated_features, axis=1, inplace=True)
test_features.drop(labels=correlated_features, axis=1, inplace=True)

train_features.shape, test_features.shape

((32190, 51), (8048, 51))

In [84]:
train_features.columns

Index(['form_field1', 'form_field2', 'form_field3', 'form_field4',
       'form_field5', 'form_field6', 'form_field7', 'form_field8',
       'form_field9', 'form_field11', 'form_field12', 'form_field13',
       'form_field14', 'form_field15', 'form_field16', 'form_field19',
       'form_field21', 'form_field22', 'form_field23', 'form_field24',
       'form_field25', 'form_field26', 'form_field28', 'form_field29',
       'form_field30', 'form_field31', 'form_field32', 'form_field33',
       'form_field34', 'form_field35', 'form_field36', 'form_field38',
       'form_field39', 'form_field40', 'form_field41', 'form_field42',
       'form_field43', 'form_field44', 'form_field45', 'form_field47',
       'form_field48', 'form_field49', 'form_field50', 'auto_any',
       'percentage_edu_auto', 'active_lines_now', 'unknowns', 'tenure_days',
       'default_status', 'cluster_clu1', 'cluster_clu2'],
      dtype='object')

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
           k_features=35,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=4)

In [ ]:
features = feature_selector.fit(np.array(train_features.fillna(0)), train_labels)

In [ ]:
filtered_features= train_features.columns[list(features.k_feature_idx_)]
filtered_features

In [99]:
X_train = train_data_[['form_field1', 'form_field2', 'form_field3', 'form_field4',
       'form_field5', 'form_field6', 'form_field7', 'form_field8',
       'form_field9', 'form_field11', 'form_field12', 'form_field13',
       'form_field14', 'form_field15', 'form_field16', 'form_field19',
       'form_field21', 'form_field22', 'form_field23', 'form_field24',
       'form_field25', 'form_field26', 'form_field28', 'form_field29',
       'form_field30', 'form_field31', 'form_field32', 'form_field33',
       'form_field34', 'form_field35', 'form_field36', 'form_field38',
       'form_field39', 'form_field40', 'form_field41', 'form_field42',
       'form_field43', 'form_field44', 'form_field45', 'form_field47',
       'form_field48', 'form_field49', 'form_field50', 'auto_any',
       'percentage_edu_auto', 'active_lines_now', 'unknowns', 'tenure_days', 'cluster_clu1', 'cluster_clu2']]
y_train = train_data_.default_status
X_test = test_data_[['form_field1', 'form_field2', 'form_field3', 'form_field4',
       'form_field5', 'form_field6', 'form_field7', 'form_field8',
       'form_field9', 'form_field11', 'form_field12', 'form_field13',
       'form_field14', 'form_field15', 'form_field16', 'form_field19',
       'form_field21', 'form_field22', 'form_field23', 'form_field24',
       'form_field25', 'form_field26', 'form_field28', 'form_field29',
       'form_field30', 'form_field31', 'form_field32', 'form_field33',
       'form_field34', 'form_field35', 'form_field36', 'form_field38',
       'form_field39', 'form_field40', 'form_field41', 'form_field42',
       'form_field43', 'form_field44', 'form_field45', 'form_field47',
       'form_field48', 'form_field49', 'form_field50', 'auto_any',
       'percentage_edu_auto', 'active_lines_now', 'unknowns', 'tenure_days', 'cluster_clu1', 'cluster_clu2']]

In [100]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [101]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [89]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [102]:
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:  2.2min finished


[16:52:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



 Time taken: 0 hours 2 minutes and 46.65 seconds.


In [103]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([24.38634348, 37.53436867, 35.52901411, 25.64213077, 33.68307122]), 'std_fit_time': array([0.05037448, 0.11875964, 0.04001505, 0.23332155, 0.18900612]), 'mean_score_time': array([0.07905555, 0.4977591 , 0.468805  , 0.28043715, 0.30451179]), 'std_score_time': array([0.00443738, 0.01275163, 0.00518452, 0.05902593, 0.00557843]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
         

In [104]:
test1 = pd.read_csv('test.csv')

In [105]:
y_test = random_search.predict_proba(X_test)
results_df = pd.DataFrame(data={'Applicant_ID':test1['Applicant_ID'], 'default_status':y_test[:,1]})
results_df.to_csv('predicted14.csv', index=False)